In [2]:
from sinexpDL import *


In [3]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
# manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

C:\Users\aashr\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Random Seed:  999


In [4]:
dataset = specGds(1_000_0, 1 / 500_0, 0.5)

# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 16

device = "cuda" if torch.cuda.is_available() else "cpu"

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True, num_workers=workers
)
test = next(iter(dataloader))
# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 2

# what the 2 inputs get turned in to in the first linear layer
nzac = 128

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

print(test.shape)

self.slist.shape = (10000,) | self.numpis = 3 | self.__len__() = 10000
torch.Size([16, 1, 64, 64])


In [5]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [17]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # turns the 'interesting and 'informative' ' 2 parameters into slightly more informative 128 parameters
        self.lin1 = nn.Linear(nz, nzac)
        # input is Z, going into a convolution
        # self.btn0 = nn.BatchNorm2d(nzac)
        self.ctp1 = nn.ConvTranspose2d(
            in_channels=nzac,
            out_channels=ngf * 8,
            kernel_size=4,
            stride=1,
            padding=0,
            bias=False,
        )
        self.btn1 = nn.BatchNorm2d(ngf * 8)
        self.relu = nn.ReLU(True)
        # state size. (ngf*8) x 4 x 4
        self.ctp2 = nn.ConvTranspose2d(
            in_channels=ngf * 8,
            out_channels=ngf * 4,
            kernel_size=4,
            stride=2,
            padding=1,
            bias=False,
        )
        self.btn2 = nn.BatchNorm2d(ngf * 4)
        # nn.ReLU(True),
        # state size. (ngf*4) x 8 x 8
        self.ctp3 = nn.ConvTranspose2d(
            in_channels=ngf * 4,
            out_channels=ngf * 2,
            kernel_size=4,
            stride=2,
            padding=1,
            bias=False,
        )
        self.btn3 = nn.BatchNorm2d(ngf * 2)
        # state size. (ngf*2) x 16 x 16
        self.ctp4 = nn.ConvTranspose2d(
            in_channels=ngf * 2,
            out_channels=ngf,
            kernel_size=4,
            stride=2,
            padding=1,
            bias=False,
        )
        self.btn4 = nn.BatchNorm2d(ngf)
        # state size. (ngf) x 32 x 32
        self.ctp5 = nn.ConvTranspose2d(
            in_channels=ngf,
            out_channels=nc,
            kernel_size=4,
            stride=2,
            padding=1,
            bias=False
        )
        self.tanh = nn.Tanh()
        # state size. (nc) x 64 x 64

    def forward(self, inp):
        inp = inp.to(self.device).float() 
        inm = self.lin1(inp)
        # inm = self.btn0(inm)

        inm = self.ctp1(inm)
        inm = self.btn1(inm)
        inm = self.relu(inm)

        inm = self.ctp2(inm)
        inm = self.btn2(inm)
        inm = self.relu(inm)
        
        inm = self.ctp3(inm)
        inm = self.btn3(inm)
        inm = self.relu(inm)
        
        inm = self.ctp4(inm)
        inm = self.btn4(inm)
        inm = self.relu(inm)
        
        inm = self.ctp5(inm)
        fin = self.relu(inm)
        
        return fin

In [18]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(
                in_channels=nc,
                out_channels=ndf,
                kernel_size=4,
                stride=2,
                padding=1,
                bias=False,
            ),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(
                in_channels=ndf,
                out_channels=ndf * 2,
                kernel_size=4,
                stride=2,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(
                in_channels=ndf * 2,
                out_channels=ndf * 4,
                kernel_size=4,
                stride=2,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(
                in_channels=ndf * 4,
                out_channels=ndf * 8,
                kernel_size=4,
                stride=2,
                padding=1,
                bias=False,
            ),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(
                in_channels=ndf * 8,
                out_channels=1,
                kernel_size=4,
                stride=1,
                padding=0,
                bias=False,
            ),
            nn.Sigmoid(),
        )

    def forward(self, input):
        input = input.to(self.device)
        return self.main(input)

In [21]:
# Create the generator
netG = Generator(ngpu).to(device)
netD = Discriminator(ngpu).to(device)

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
netG.apply(weights_init)
netD.apply(weights_init)

# Print the model
print(netG)
print(netD)
inp = torch.randn(16,1,nz,1)
print(inp.shape)
netG(inp)

Generator(
  (lin1): Linear(in_features=2, out_features=128, bias=True)
  (btn0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ctp1): ConvTranspose2d(128, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (btn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (ctp2): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (btn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ctp3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (btn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ctp4): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (btn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ctp5): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x1 and 2x128)

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


In [ ]:
fixed_noise.shape

torch.Size([64, 2, 1, 1])